In [99]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 19 14:54:31 2018

@author: 6Zhilins
"""
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import time

#1.1. Laden der Daten:
d = np.load('./trainingsDaten.npz')
train_imgs = d['data']
train_labels = d['labels']

t = np.load('./validierungsDaten.npz')
test_imgs = t['data']
test_labels = t['labels']

#Erstes Bild und Label auslesen:
img1 = train_imgs[0,:,:] 
label1 = train_labels[0] 

#1.2. Mittelwert jedes Bildes

train_data_mean = np.mean(tr_imgs,axis=(2,1))
test_data_mean = np.mean(test_imgs,axis=(2,1))

In [539]:
#1.3. 
def predict_mean(train,test,train_labels):
    # Ein Array Umwandeln in 2D und dabei die erste Reihe wiederholen.
    test_matrix = np.tile(test,([len(train),1])).T
    # Euklidische Distanz ausrechnen (Absolute umgeht Wurzel und Quadrat)
    test_distance_matrix = np.absolute(test_matrix - train)
    # Indice der kleinsten Werte anzeigen lassen
    indices = np.argmin(test_distance_matrix,1) 
    # Werte der Indice ausgeben
    
    return np.take(train_labels, indices)

In [540]:
#1.4.
prediction_labels =  predict_mean(train_data_mean,test_data_mean,train_labels)
evaluation = compare_train_test(prediction_labels,test_labels)
np.mean(prediction_labels == test_labels)

# Das Ergebnis ist inakzeptabel da es größer als 1/3 sein sollte.

0.2

In [542]:
#1.5
train_data_std = np.std(tr_imgs,axis=(2,1))
test_data_std = np.std(test_imgs,axis=(2,1))
train_mean_std = np.column_stack((train_data_mean,train_data_std))
test_mean_std = np.column_stack((test_data_mean,test_data_std))

def predict_mean_std(train,test,train_labels):
    test_matrix = np.tile(test, (30,2,1,))
    train_matrix = np.tile(train,(len(test))).reshape((30, 60, 2))
    # Euklidische Distanz ausrechnen (Absolute umgeht Wurzel und Quadrat)
    test_distance_matrix = np.absolute(test_matrix-train_matrix).sum(axis=2)
    # Indice der kleinsten Werte anzeigen lassen
    indices = np.argmin(test_distance_matrix,1) 
    # Werte der Indice ausgeben
    prediction_labels = np.take(train_labels, indices)
    return prediction_labels

prediction_labels = predict_mean_std(train_mean_std, test_mean_std, train_labels)
np.mean(prediction_labels == test_labels)
# Das Ergebnis ist besser da es größer als 1/3 ist.

0.43333333333333335

In [491]:
2.
#This function is able to return one of eight different matrix norms, 
#or one of an infinite number of vector norms (described below), 
#depending on the value of the ord parameter.
#None -> Frobenius norm:
#The Frobenius norm, sometimes also called the Euclidean norm
#(a term unfortunately also used for the vector -norm),
#is matrix norm of an matrix defined as the square root
#of the sum of the absolute squares of its elements,
#(Golub and van Loan 1996, p. 55).

def predict_labels(measure_of_training_data, measure_of_test_data, train_labels,test_labels):
    results = []
    for test_img in measure_of_test_data:
        results.append([np.linalg.norm(test_img,train_img) for train_img in measure_of_training_data])

    indices = np.argmin(results, 1)
    predicted_labels = [train_labels[x] for x in indices]
    return predicted_labels

hist_of_training_data = [np.histogram(img, bins = 8, range = (0,256))[0] for img in tr_imgs]
hist_of_test_data = [np.histogram(img, bins = 8, range = (0,256))[0] for img in test_imgs]
predicted_labels2 = predict_labels(hist_of_training_data, hist_of_test_data,  tr_labels,test_labels)

np.mean(predicted_labels2 == test_labels)

success = 0

for bin_num in range(1,256):
    hist_of_training_data = [np.histogram(img, bins = bin_num, range = (0,256))[0] for img in tr_imgs]
    hist_of_test_data = [np.histogram(img, bins = bin_num, range = (0,256))[0] for img in test_imgs]
    predicted_labels = predict_labels(hist_of_training_data, hist_of_test_data,  tr_labels,test_labels)
    rate = np.mean(predicted_labels == test_labels)
    if rate > success:
        success = rate
        success_bin = bin_num
        prediction = predicted_labels

In [494]:
print(success,bin_num)

0.5666666666666667 255


In [496]:
#3.
con_matrix = confusion_matrix(tr_labels, prediction2)
con_matrix_df = pd.DataFrame(con_matrix, columns=[['Autos',"Hirsch","Schiff"]])
con_matrix_df.index = ['Autos',"Hirsch","Schiff"]
print(con_matrix_df)

       Autos Hirsch Schiff
Autos     16      1      3
Hirsch     6      7      7
Schiff     9      4      7


In [532]:
from sklearn.neighbors import KNeighborsClassifier

X = train_imgs
y = tr_labels
neigh = KNeighborsClassifier(n_neighbors=5)
dataset_size = len(X)
X = X.reshape(dataset_size,-1)
neigh.fit(X, y) # doctest: +ELLIPSIS

X0 = test_imgs
dataset_size = len(X0)
X0 = X0.reshape(dataset_size,-1)
prediction = neigh.predict(X0)

In [543]:
%%timeit
predict_mean(train_data_mean,test_data_mean,train_labels)

25.8 µs ± 669 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [545]:
%%timeit
predict_mean_std(train_mean_std, test_mean_std, train_labels)

84.3 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [549]:
%%timeit
results = []
for train_img in train_data_mean:
    results.append([euclidian_distance(train_img,b) for b in test_mean_std])

minima = np.argmin(results,1)
predicted_labels = [test_labels[x] for x in minima]

12.5 ms ± 312 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [550]:
#1.3.



## REFRACTORED
def euclidian_distance(a,b):
    return np.linalg.norm(a-b)
v_euclidian_distance = np.vectorize(euclidian_distance)
def get_euclidian_distance(a,b):
    return v_euclidian_distance(a,b)
ed_arr = np.apply_along_axis(get_euclidian_distance,0,a_tile,b=b)
indices = np.argmin(ed_arr,1)
np.take(b, indices)

def compare_train_test(predicted_labels,test_labels):
    evaluation = []
    for i,test_label in enumerate(test_labels):
        evaluation.append(test_label == predicted_labels[i])
    return evaluation

ValueError: operands could not be broadcast together with shapes (5,) (7,) 

In [ ]:
def compare_train_test(predicted_labels, tr_labels,test_labels):
    evaluation = []
    for i,tr_label in enumerate(tr_labels):
        evaluation.append(tr_label == predicted_labels[i])
    return evaluation
    
prediction = predict_labels(median_of_training_data,median_of_test_data,  tr_labels,test_labels)
evaluation = compare_train_test(prediction, tr_labels,test_labels)
print("Accuracy: Total Images "+str(len(tr_labels))+". Correct predicted:"+\
str(np.sum(evalution)) + " Percentage: " + str((float(np.sum(evalution))/len(tr_labels))*100) +"%")
tick = time.time() #Timer ist gestoppt
tick_tock=tick-tock #bereche ob wie viele Zeit hat das geläuft
print()

# Ab 75% würde es zuverlässig sein? Statistisch akkurat relevant wäre es ab 95%
#2.
hist_of_training_data = [np.histogram(img, bins = 8, range = (0,256))[0] for img in tr_imgs]
hist_of_test_data = [np.histogram(img, bins = 8, range = (0,256))[0] for img in test_imgs]
prediction2 = predict_labels(hist_of_training_data,hist_of_test_data,  tr_labels,test_labels)

evaluation2 = compare_train_test(prediction2,  tr_labels,test_labels)
print("Accuracy: Total Images "+str(len(tr_labels))+". Correct predicted:"+\
str(np.sum(evaluation2)) + " Percentage: " + str((float(np.sum(evaluation2))/len(tr_labels))*100) +"%")
#3.
tock = time.time() #Timer on
con_matrix = confusion_matrix(tr_labels, prediction2)
con_matrix_df = pd.DataFrame(con_matrix, columns=[['Autos',"Hirsch","Schiff"]])
con_matrix_df.index = ['Autos',"Hirsch","Schiff"]
print(con_matrix_df)

tick = time.time() #Timer off
tick_tock2=tick-tock #berechen ob wie viele Zeit hat das geläuft

#bereche ob wann schnelle war
if tick_tock<tick_tock2:
    print("Erstes Mal war schnelle für " + str(tick_tock2-tick_tock) + " sec")
else:
    print("Zweites Mal war schnelle für " + str(tick_tock-tick_tock2) + " sec")